In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import os 

In [2]:
# check if directory exists to save results, if it does not exist, create it
path_str = "../data/LigaMX"
if(os.path.exists(path_str)):
    print(path_str + " exists")
else:
    print(path_str + " does not exists amd will be created.")
    os.mkdir(path_str)

../data/LigaMX does not exists amd will be created.


In [3]:


# Puedes cambiar este por el link de cualquier otra liga siempre y cuando se de la ruta /todos_partidos
url = "https://www.livefutbol.com/todos_partidos/mex-primera-division-2023-2024-clausura/"

response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

select_element = soup.find('select', {'name': 'saison'})

urls = ['https://www.livefutbol.com' + option['value'] for option in select_element.find_all('option')]

temporada = [option['value'] for option in select_element.find_all('option')]

matches_df =[]
for url in urls:
   
    nombre = url.split('-')[-1].strip('/')
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    matches = []

    for row in soup.find_all("tr"):
        cols = row.find_all("td")
        if len(cols) > 5:  
            match = {
                "date": cols[0].get_text(strip=True),
                "time": cols[1].get_text(strip=True),
                "torneo": nombre,
                "team_home": cols[2].get_text(strip=True),
                "team_away": cols[4].get_text(strip=True),
                "score": cols[5].get_text(strip=True)
            }
            matches.append(match)
            
    matches = pd.DataFrame.from_dict(matches, orient = 'columns')
    matches = matches[matches['score']!='-:-']
    
    for i, row in matches.iterrows():
        try:
            final, halftime = row['score'].strip().split(" ")
            final_home, final_away = final.split(':')
            ht_home, ht_away = halftime.strip('()').split(':')

            matches.loc[i, 'final_score_home'] = int(final_home)
            matches.loc[i, 'final_score_away'] = int(final_away)
            matches.loc[i, 'halftime_score_home'] = int(ht_home)
            matches.loc[i, 'halftime_score_away'] = int(ht_away)
        except Exception as e:
            try:
                final = row['score'].strip()
                final_home, final_away = final.split(':')
                matches.loc[i, 'final_score_home'] = int(final_home)
                matches.loc[i, 'final_score_away'] = int(final_away)
            except:   
                matches.loc[i, 'final_score_home'] = None
                matches.loc[i, 'final_score_away'] = None
                matches.loc[i, 'halftime_score_home'] = None
                matches.loc[i, 'halftime_score_away'] = None
        
    matches_df.append(matches)
    matches_df = pd.concat(matches_df)
    temporada = url.split('/')
    save_str = path_str + "/" + temporada[4] + ".csv"
    matches_df.to_csv(save_str)
    print([" saving save_str..."  + save_str + " done"])
    matches_df =[]

[' saving save_str...../data/LigaMX/mex-primera-division-2023-2024-clausura-playoffs.csv done']
[' saving save_str...../data/LigaMX/mex-primera-division-2023-2024-clausura.csv done']
[' saving save_str...../data/LigaMX/mex-primera-division-2023-2024-apertura-playoffs.csv done']
[' saving save_str...../data/LigaMX/mex-primera-division-2023-2024-apertura.csv done']
[' saving save_str...../data/LigaMX/mex-primera-division-2022-2023-clausura-playoffs.csv done']
[' saving save_str...../data/LigaMX/mex-primera-division-2022-2023-clausura.csv done']
[' saving save_str...../data/LigaMX/mex-primera-division-2022-2023-apertura-playoffs.csv done']
[' saving save_str...../data/LigaMX/mex-primera-division-2022-2023-apertura.csv done']
[' saving save_str...../data/LigaMX/mex-primera-division-2021-2022-clausura-playoffs.csv done']
[' saving save_str...../data/LigaMX/mex-primera-division-2021-2022-clausura.csv done']
[' saving save_str...../data/LigaMX/mex-primera-division-2021-2022-apertura-playoffs.